In [ ]:
pip install pyomo

In [ ]:
!apt-get install -y glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
people = pd.read_csv('people.csv')
flights = pd.read_csv('flights.csv')

In [ ]:
people = people.drop('Presenter', axis=1)
people.head()

,Name,Level,Office,Bill Rate,Departing Airport,Average Market Price,Primary Key
0,Aaron Clark,Senior Associate,"Phoenix, AZ",330.0,PHX,159.30,286362.0
1,Abel Harvey,Senior Associate,"New York, NY",405.0,JFK,198.27,327560.0
2,Abel Patrick,Associate,"San Antonio, TX",290.0,SAT,182.92,432240.0
3,Ada Lawrence,Senior Associate,"Oklahoma City, OK",325.0,OKC,180.94,697141.0
4,Ada McCormick,Associate,"Cedar Rapids/Iowa City, IA",270.0,CID,207.73,192682.0


In [ ]:
flights.head()

,Primary Key(Fi),Departing Airport,Destination Airport,Avg_MARKET_FARE (Ci),Departure Time,Arrival Time,Dtime,Atime,Duration
0,6042,DAL,ATL,182.92,652,948,06:52:00,09:48:00,2.933333
1,6043,DAL,ATL,182.92,653,930,06:53:00,09:30:00,2.616667
2,6044,DAL,ATL,182.92,653,931,06:53:00,09:31:00,2.633333
3,6045,DAL,ATL,182.92,653,934,06:53:00,09:34:00,2.683333
4,6046,DAL,ATL,182.92,653,936,06:53:00,09:36:00,2.716667


In [ ]:
# Get a unique list of arrival airports
arrival_airports = flights['Destination Airport'].unique()

# Add columns for each arrival airport to the people dataframe
for airport in arrival_airports:
    people[airport] = None  # Initialize columns with None

# Calculate minimum cost for each arrival airport
for idx, person in people.iterrows():
    # Filter flights for the person's departing airport
    available_flights = flights[flights['Departing Airport'] == person['Departing Airport']]

    for airport in arrival_airports:
        # Filter flights to the specific arrival airport
        flights_to_airport = available_flights[available_flights['Destination Airport'] == airport]

        if not flights_to_airport.empty:
            # Calculate the cost for each flight
            flights_to_airport['cost'] = person['Bill Rate'] * flights_to_airport['Duration'] + flights_to_airport['Avg_MARKET_FARE (Ci)']

            # Find the minimum cost
            min_cost = flights_to_airport['cost'].min()
        else:
            # If no flights to this airport, set cost as None
            min_cost = None

        # Update the person's column for the arrival airport
        people.at[idx, airport] = min_cost

# Output the updated dataframe
people.head(10)

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_to_airport['cost'] = person['Bill Rate'] * flights_to_airport['Duration'] + flights_to_airport['Avg_MARKET_FARE (Ci)']
<ipython-input-7-417e06a090ed>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_to_airport['cost'] = person['Bill Rate'] * flights_to_airport['Duration'] + flights_to_airport['Avg_MARKET_FARE (Ci)']
<ipython-input-7-417e06a090ed>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,Name,Level,Office,Bill Rate,Departing Airport,Average Market Price,Primary Key,ATL,BNA,ORD,MCO,LAS,DFW
0,Aaron Clark,Senior Associate,"Phoenix, AZ",330.0,PHX,159.30,286362.0,1796.24,1456.2,1472.81,1945.16,144.63,1075.77
1,Abel Harvey,Senior Associate,"New York, NY",405.0,JFK,198.27,327560.0,900.5,559.62,586.02,1038.66,1021.77,1019.28
2,Abel Patrick,Associate,"San Antonio, TX",290.0,SAT,182.92,432240.0,1075.2,775.533333,-1402.24,1213.776667,395.586667,452.216667
3,Ada Lawrence,Senior Associate,"Oklahoma City, OK",325.0,OKC,180.94,697141.0,1004.51,None,762.446667,1280.523333,283.856667,446.156667
4,Ada McCormick,Associate,"Cedar Rapids/Iowa City, IA",270.0,CID,207.73,192682.0,856.61,487.61,406.61,1179.52,455.23,602.66
5,Adam Gallegos,Senior Associate,"Charlotte, NC",330.0,CLT,153.22,413803.0,357.8,181.11,374.99,584.95,612.95,584.4
6,Adam Ingram,Supervisor,"St. Louis, MO",420.0,STL,133.82,479418.0,1119.66,606.81,533.02,1369.97,623.82,827.23
7,Adam Neal,Associate,"Philadelphia, PA",270.0,PHL,166.34,282940.0,604.3,373.03,413.21,-1440.16,679.34,679.34
8,Adam Padilla,Associate,"Chicago, IL",290.0,ORD,155.39,278831.0,861.436667,498.16,None,1083.64,502.076667,642.036667
9,Adam Stevens,Senior Associate,"Minneapolis, MN",330.0,MSP,83.13,390764.0,1156.87,722.77,497.36,1412.69,509.83,835.64


In [ ]:
# Fill nulls with 0 if departing airport is the same as the column name
for col in arrival_airports:
    people.loc[people['Departing Airport'] == col, col] = people.loc[people['Departing Airport'] == col, col].fillna(0)

people.head(10)

<ipython-input-8-ed18905223cb>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  people.loc[people['Departing Airport'] == col, col] = people.loc[people['Departing Airport'] == col, col].fillna(0)
<ipython-input-8-ed18905223cb>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  people.loc[people['Departing Airport'] == col, col] = people.loc[people['Departing Airport'] == col, col].fillna(0)
<ipython-input-8-ed18905223cb>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call resu

,Name,Level,Office,Bill Rate,Departing Airport,Average Market Price,Primary Key,ATL,BNA,ORD,MCO,LAS,DFW
0,Aaron Clark,Senior Associate,"Phoenix, AZ",330.0,PHX,159.30,286362.0,1796.24,1456.2,1472.81,1945.16,144.63,1075.77
1,Abel Harvey,Senior Associate,"New York, NY",405.0,JFK,198.27,327560.0,900.5,559.62,586.02,1038.66,1021.77,1019.28
2,Abel Patrick,Associate,"San Antonio, TX",290.0,SAT,182.92,432240.0,1075.2,775.533333,-1402.24,1213.776667,395.586667,452.216667
3,Ada Lawrence,Senior Associate,"Oklahoma City, OK",325.0,OKC,180.94,697141.0,1004.51,None,762.446667,1280.523333,283.856667,446.156667
4,Ada McCormick,Associate,"Cedar Rapids/Iowa City, IA",270.0,CID,207.73,192682.0,856.61,487.61,406.61,1179.52,455.23,602.66
5,Adam Gallegos,Senior Associate,"Charlotte, NC",330.0,CLT,153.22,413803.0,357.8,181.11,374.99,584.95,612.95,584.4
6,Adam Ingram,Supervisor,"St. Louis, MO",420.0,STL,133.82,479418.0,1119.66,606.81,533.02,1369.97,623.82,827.23
7,Adam Neal,Associate,"Philadelphia, PA",270.0,PHL,166.34,282940.0,604.3,373.03,413.21,-1440.16,679.34,679.34
8,Adam Padilla,Associate,"Chicago, IL",290.0,ORD,155.39,278831.0,861.436667,498.16,0,1083.64,502.076667,642.036667
9,Adam Stevens,Senior Associate,"Minneapolis, MN",330.0,MSP,83.13,390764.0,1156.87,722.77,497.36,1412.69,509.83,835.64


In [ ]:
# Fill nulls with 0 if column name is DFW and departing airport is DAL
people.loc[(people['Departing Airport'] == 'DAL') & (people['DFW'].isnull()), 'DFW'] = 0
people.head(30)

,Name,Level,Office,Bill Rate,Departing Airport,Average Market Price,Primary Key,ATL,BNA,ORD,MCO,LAS,DFW
0,Aaron Clark,Senior Associate,"Phoenix, AZ",330.0,PHX,159.30,286362.0,1796.24,1456.2,1472.81,1945.16,144.63,1075.77
1,Abel Harvey,Senior Associate,"New York, NY",405.0,JFK,198.27,327560.0,900.5,559.62,586.02,1038.66,1021.77,1019.28
2,Abel Patrick,Associate,"San Antonio, TX",290.0,SAT,182.92,432240.0,1075.2,775.533333,-1402.24,1213.776667,395.586667,452.216667
3,Ada Lawrence,Senior Associate,"Oklahoma City, OK",325.0,OKC,180.94,697141.0,1004.51,None,762.446667,1280.523333,283.856667,446.156667
4,Ada McCormick,Associate,"Cedar Rapids/Iowa City, IA",270.0,CID,207.73,192682.0,856.61,487.61,406.61,1179.52,455.23,602.66
5,Adam Gallegos,Senior Associate,"Charlotte, NC",330.0,CLT,153.22,413803.0,357.8,181.11,374.99,584.95,612.95,584.4
6,Adam Ingram,Supervisor,"St. Louis, MO",420.0,STL,133.82,479418.0,1119.66,606.81,533.02,1369.97,623.82,827.23
7,Adam Neal,Associate,"Philadelphia, PA",270.0,PHL,166.34,282940.0,604.3,373.03,413.21,-1440.16,679.34,679.34
8,Adam Padilla,Associate,"Chicago, IL",290.0,ORD,155.39,278831.0,861.436667,498.16,0,1083.64,502.076667,642.036667
9,Adam Stevens,Senior Associate,"Minneapolis, MN",330.0,MSP,83.13,390764.0,1156.87,722.77,497.36,1412.69,509.83,835.64


In [ ]:
# Fill nulls in the last 6 columns with 1000000
last_six_columns = people.columns[-6:]
people[last_six_columns] = people[last_six_columns].fillna(1000000)

# Output the updated dataframe
people.head(10)

<ipython-input-10-3af79cfa41ac>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  people[last_six_columns] = people[last_six_columns].fillna(1000000)


,Name,Level,Office,Bill Rate,Departing Airport,Average Market Price,Primary Key,ATL,BNA,ORD,MCO,LAS,DFW
0,Aaron Clark,Senior Associate,"Phoenix, AZ",330.0,PHX,159.30,286362.0,1796.240000,1456.200000,1472.810000,1945.160000,144.630000,1075.770000
1,Abel Harvey,Senior Associate,"New York, NY",405.0,JFK,198.27,327560.0,900.500000,559.620000,586.020000,1038.660000,1021.770000,1019.280000
2,Abel Patrick,Associate,"San Antonio, TX",290.0,SAT,182.92,432240.0,1075.200000,775.533333,-1402.240000,1213.776667,395.586667,452.216667
3,Ada Lawrence,Senior Associate,"Oklahoma City, OK",325.0,OKC,180.94,697141.0,1004.510000,1000000.000000,762.446667,1280.523333,283.856667,446.156667
4,Ada McCormick,Associate,"Cedar Rapids/Iowa City, IA",270.0,CID,207.73,192682.0,856.610000,487.610000,406.610000,1179.520000,455.230000,602.660000
5,Adam Gallegos,Senior Associate,"Charlotte, NC",330.0,CLT,153.22,413803.0,357.800000,181.110000,374.990000,584.950000,612.950000,584.400000
6,Adam Ingram,Supervisor,"St. Louis, MO",420.0,STL,133.82,479418.0,1119.660000,606.810000,533.020000,1369.970000,623.820000,827.230000
7,Adam Neal,Associate,"Philadelphia, PA",270.0,PHL,166.34,282940.0,604.300000,373.030000,413.210000,-1440.160000,679.340000,679.340000
8,Adam Padilla,Associate,"Chicago, IL",290.0,ORD,155.39,278831.0,861.436667,498.160000,0.000000,1083.640000,502.076667,642.036667
9,Adam Stevens,Senior Associate,"Minneapolis, MN",330.0,MSP,83.13,390764.0,1156.870000,722.770000,497.360000,1412.690000,509.830000,835.640000


In [ ]:
people.isnull().sum()

,0
Name,0
Level,0
Office,0
Bill Rate,0
Departing Airport,0
Average Market Price,0
Primary Key,23
ATL,0
BNA,0
ORD,0


In [ ]:
# Rename the last 6 columns by adding 1 to their names
last_six_columns = people.columns[-6:]
new_column_names = [col + '1' for col in last_six_columns]
people = people.rename(columns=dict(zip(last_six_columns, new_column_names)))

# Make 5 copies of the last 6 columns
for i in range(2, 6):  # Create copies for numbers 2 through 5
    last_six_columns = [col for col in people.columns if col.endswith(str(i-1))]
    new_column_names = [col[:-1] + str(i) for col in last_six_columns]
    people = pd.concat([people, people[last_six_columns].rename(columns=dict(zip(last_six_columns, new_column_names)))], axis=1)
people.head()

,Name,Level,Office,Bill Rate,Departing Airport,Average Market Price,Primary Key,ATL1,BNA1,ORD1,...,ORD4,MCO4,LAS4,DFW4,ATL5,BNA5,ORD5,MCO5,LAS5,DFW5
0,Aaron Clark,Senior Associate,"Phoenix, AZ",330.0,PHX,159.30,286362.0,1796.24,1456.200000,1472.810000,...,1472.810000,1945.160000,144.630000,1075.770000,1796.24,1456.200000,1472.810000,1945.160000,144.630000,1075.770000
1,Abel Harvey,Senior Associate,"New York, NY",405.0,JFK,198.27,327560.0,900.50,559.620000,586.020000,...,586.020000,1038.660000,1021.770000,1019.280000,900.50,559.620000,586.020000,1038.660000,1021.770000,1019.280000
2,Abel Patrick,Associate,"San Antonio, TX",290.0,SAT,182.92,432240.0,1075.20,775.533333,-1402.240000,...,-1402.240000,1213.776667,395.586667,452.216667,1075.20,775.533333,-1402.240000,1213.776667,395.586667,452.216667
3,Ada Lawrence,Senior Associate,"Oklahoma City, OK",325.0,OKC,180.94,697141.0,1004.51,1000000.000000,762.446667,...,762.446667,1280.523333,283.856667,446.156667,1004.51,1000000.000000,762.446667,1280.523333,283.856667,446.156667
4,Ada McCormick,Associate,"Cedar Rapids/Iowa City, IA",270.0,CID,207.73,192682.0,856.61,487.610000,406.610000,...,406.610000,1179.520000,455.230000,602.660000,856.61,487.610000,406.610000,1179.520000,455.230000,602.660000


In [ ]:
people.to_csv('people_flights.csv', index=False)

In [ ]:
people_final = people.iloc[:, 7:].copy()
people_final.head()

,ATL1,BNA1,ORD1,MCO1,LAS1,DFW1,ATL2,BNA2,ORD2,MCO2,...,ORD4,MCO4,LAS4,DFW4,ATL5,BNA5,ORD5,MCO5,LAS5,DFW5
0,1796.24,1456.200000,1472.810000,1945.160000,144.630000,1075.770000,1796.24,1456.200000,1472.810000,1945.160000,...,1472.810000,1945.160000,144.630000,1075.770000,1796.24,1456.200000,1472.810000,1945.160000,144.630000,1075.770000
1,900.50,559.620000,586.020000,1038.660000,1021.770000,1019.280000,900.50,559.620000,586.020000,1038.660000,...,586.020000,1038.660000,1021.770000,1019.280000,900.50,559.620000,586.020000,1038.660000,1021.770000,1019.280000
2,1075.20,775.533333,-1402.240000,1213.776667,395.586667,452.216667,1075.20,775.533333,-1402.240000,1213.776667,...,-1402.240000,1213.776667,395.586667,452.216667,1075.20,775.533333,-1402.240000,1213.776667,395.586667,452.216667
3,1004.51,1000000.000000,762.446667,1280.523333,283.856667,446.156667,1004.51,1000000.000000,762.446667,1280.523333,...,762.446667,1280.523333,283.856667,446.156667,1004.51,1000000.000000,762.446667,1280.523333,283.856667,446.156667
4,856.61,487.610000,406.610000,1179.520000,455.230000,602.660000,856.61,487.610000,406.610000,1179.520000,...,406.610000,1179.520000,455.230000,602.660000,856.61,487.610000,406.610000,1179.520000,455.230000,602.660000


In [ ]:
#drop the columns ORD5 and BNA5 from people_final
people_final = people_final.drop(['ORD5', 'BNA5'], axis=1)

In [ ]:
conference = pd.read_csv('conference_location.csv', index_col=0)
conference.head()

,Capacity,FixedCost
Center,,
LAS1,2300,492808
LAS2,1700,538608
LAS3,2500,327928
LAS4,1850,305944
LAS5,2000,426856


In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Binary, SolverFactory, summation, minimize, Set

transport_costs = people_final  # Offices (rows) x Conference centers (columns)
fixed_costs = conference['FixedCost']
capacities = conference['Capacity']

# Sets
offices = transport_costs.index.tolist()
centers = transport_costs.columns.tolist()

# Parameters
c = {(i, j): transport_costs.loc[i, j] for i in offices for j in centers}
#f = {j: fixed_costs.loc[j, 'FixedCost'] for j in centers}
f = {j: fixed_costs.loc[j] for j in centers}

# Model definition
model = ConcreteModel()

# Decision variables
model.x = Var(offices, centers, domain=Binary)  # Assignment variable
model.y = Var(centers, domain=Binary)           # Selection variable

# Objective function
model.objective = Objective(
    expr=summation(f, model.y) + summation(c, model.x),
    sense=minimize
)

# Constraints
# Each office is assigned to exactly one center
def assignment_rule(model, i):
    return sum(model.x[i, j] for j in centers) == 1
model.assignment = Constraint(offices, rule=assignment_rule)

# A center must be open if any office is assigned to it
def center_open_rule(model, j):
    return sum(model.x[i, j] for i in offices) <= len(offices) * model.y[j]
model.center_open = Constraint(centers, rule=center_open_rule)

# Only one center is selected
def single_center_rule(model):
    return sum(model.y[j] for j in centers) == 1
model.single_center = Constraint(rule=single_center_rule)

# A center can be selected only if its capacity is greater than 1831
def capacity_selection_rule(model, j):
    return model.y[j] <= (1 if capacities[j] > 1831 else 0)
model.capacity_selection = Constraint(centers, rule=capacity_selection_rule)

In [ ]:
# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpbg4qkkfs.glpk.raw --wglp /tmp/tmpr6nl6fps.glpk.glp --cpxlp
 /tmp/tmpt9jsc8no.pyomo.lp
Reading problem data from '/tmp/tmpt9jsc8no.pyomo.lp'...
/tmp/tmpt9jsc8no.pyomo.lp:209283: warning: lower bound of variable 'x2' redefined
/tmp/tmpt9jsc8no.pyomo.lp:209283: warning: upper bound of variable 'x2' redefined
1889 rows, 51324 columns, 102676 non-zeros
51324 integer variables, all of which are binary
260607 lines were read
Writing problem data to '/tmp/tmpr6nl6fps.glpk.glp'...
207387 lines were written
GLPK Integer Optimizer 5.0
1889 rows, 51324 columns, 102676 non-zeros
51324 integer variables, all of which are binary
Preprocessing...
1853 rows, 36660 columns, 73320 non-zeros
36660 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.832e+03  ratio =  1.832e+03
GM: min|aij| =  9.711e-01  max|aij| =  1.030e+00  ratio =  1.060e+00
EQ: min|aij| =  9.430e-01  ma

In [ ]:
# Display results
print("Optimal Objective Value:", model.objective())

Optimal Objective Value: 12565257.144842772


In [ ]:
# Display selected center
selected_centers = [j for j in centers if model.y[j].value == 1]
print("Selected Airport:", selected_centers)

Selected Airport: ['ATL3']


In [ ]:
# Display assignments
assignments = {(i, j): model.x[i, j].value for i in offices for j in centers if model.x[i, j].value > 0.5}
print("Assignments:")
for (office, center), value in assignments.items():
    print(f"{office} -> {center}")

Assignments:
0 -> ATL3
1 -> ATL3
2 -> ATL3
3 -> ATL3
4 -> ATL3
5 -> ATL3
6 -> ATL3
7 -> ATL3
8 -> ATL3
9 -> ATL3
10 -> ATL3
11 -> ATL3
12 -> ATL3
13 -> ATL3
14 -> ATL3
15 -> ATL3
16 -> ATL3
17 -> ATL3
18 -> ATL3
19 -> ATL3
20 -> ATL3
21 -> ATL3
22 -> ATL3
23 -> ATL3
24 -> ATL3
25 -> ATL3
26 -> ATL3
27 -> ATL3
28 -> ATL3
29 -> ATL3
30 -> ATL3
31 -> ATL3
32 -> ATL3
33 -> ATL3
34 -> ATL3
35 -> ATL3
36 -> ATL3
37 -> ATL3
38 -> ATL3
39 -> ATL3
40 -> ATL3
41 -> ATL3
42 -> ATL3
43 -> ATL3
44 -> ATL3
45 -> ATL3
46 -> ATL3
47 -> ATL3
48 -> ATL3
49 -> ATL3
50 -> ATL3
51 -> ATL3
52 -> ATL3
53 -> ATL3
54 -> ATL3
55 -> ATL3
56 -> ATL3
57 -> ATL3
58 -> ATL3
59 -> ATL3
60 -> ATL3
61 -> ATL3
62 -> ATL3
63 -> ATL3
64 -> ATL3
65 -> ATL3
66 -> ATL3
67 -> ATL3
68 -> ATL3
69 -> ATL3
70 -> ATL3
71 -> ATL3
72 -> ATL3
73 -> ATL3
74 -> ATL3
75 -> ATL3
76 -> ATL3
77 -> ATL3
78 -> ATL3
79 -> ATL3
80 -> ATL3
81 -> ATL3
82 -> ATL3
83 -> ATL3
84 -> ATL3
85 -> ATL3
86 -> ATL3
87 -> ATL3
88 -> ATL3
89 -> ATL3
90 -> A

In [ ]:
#drop the ATL3 column from people_final
people_final = people_final.drop('ATL3', axis=1)

#drop the ATL3 row from conference
conference = conference.drop('ATL3', axis=0)

In [ ]:
transport_costs = people_final  # Offices (rows) x Conference centers (columns)
fixed_costs = conference['FixedCost']
capacities = conference['Capacity']

# Sets
offices = transport_costs.index.tolist()
centers = transport_costs.columns.tolist()

# Parameters
c = {(i, j): transport_costs.loc[i, j] for i in offices for j in centers}
#f = {j: fixed_costs.loc[j, 'FixedCost'] for j in centers}
f = {j: fixed_costs.loc[j] for j in centers}

# Model definition
model = ConcreteModel()

# Decision variables
model.x = Var(offices, centers, domain=Binary)  # Assignment variable
model.y = Var(centers, domain=Binary)           # Selection variable

# Objective function
model.objective = Objective(
    expr=summation(f, model.y) + summation(c, model.x),
    sense=minimize
)

# Constraints
# Each office is assigned to exactly one center
def assignment_rule(model, i):
    return sum(model.x[i, j] for j in centers) == 1
model.assignment = Constraint(offices, rule=assignment_rule)

# A center must be open if any office is assigned to it
def center_open_rule(model, j):
    return sum(model.x[i, j] for i in offices) <= len(offices) * model.y[j]
model.center_open = Constraint(centers, rule=center_open_rule)

# Only one center is selected
def single_center_rule(model):
    return sum(model.y[j] for j in centers) == 1
model.single_center = Constraint(rule=single_center_rule)

# A center can be selected only if its capacity is greater than 1831
def capacity_selection_rule(model, j):
    return model.y[j] <= (1 if capacities[j] > 1831 else 0)
model.capacity_selection = Constraint(centers, rule=capacity_selection_rule)

In [ ]:
# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpyoegbncc.glpk.raw --wglp /tmp/tmppdsjckya.glpk.glp --cpxlp
 /tmp/tmpdx48dwg8.pyomo.lp
Reading problem data from '/tmp/tmpdx48dwg8.pyomo.lp'...
/tmp/tmpdx48dwg8.pyomo.lp:201971: warning: lower bound of variable 'x2' redefined
/tmp/tmpdx48dwg8.pyomo.lp:201971: warning: upper bound of variable 'x2' redefined
1887 rows, 49491 columns, 99009 non-zeros
49491 integer variables, all of which are binary
251462 lines were read
Writing problem data to '/tmp/tmppdsjckya.glpk.glp'...
200077 lines were written
GLPK Integer Optimizer 5.0
1887 rows, 49491 columns, 99009 non-zeros
49491 integer variables, all of which are binary
Preprocessing...
1852 rows, 34827 columns, 69654 non-zeros
34827 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.832e+03  ratio =  1.832e+03
GM: min|aij| =  9.711e-01  max|aij| =  1.030e+00  ratio =  1.060e+00
EQ: min|aij| =  9.430e-01  max|

In [ ]:
# Display results
print("Optimal Objective Value:", model.objective())

# Display selected center
selected_centers = [j for j in centers if model.y[j].value == 1]
print("Selected Airport:", selected_centers)

Optimal Objective Value: 12574417.144842772
Selected Airport: ['ATL4']


In [ ]:
#drop the ATL4 column from people_final
people_final = people_final.drop('ATL4', axis=1)

#drop the ATL4 row from conference
conference = conference.drop('ATL4', axis=0)

In [ ]:
transport_costs = people_final  # Offices (rows) x Conference centers (columns)
fixed_costs = conference['FixedCost']
capacities = conference['Capacity']

# Sets
offices = transport_costs.index.tolist()
centers = transport_costs.columns.tolist()

# Parameters
c = {(i, j): transport_costs.loc[i, j] for i in offices for j in centers}
#f = {j: fixed_costs.loc[j, 'FixedCost'] for j in centers}
f = {j: fixed_costs.loc[j] for j in centers}

# Model definition
model = ConcreteModel()

# Decision variables
model.x = Var(offices, centers, domain=Binary)  # Assignment variable
model.y = Var(centers, domain=Binary)           # Selection variable

# Objective function
model.objective = Objective(
    expr=summation(f, model.y) + summation(c, model.x),
    sense=minimize
)

# Constraints
# Each office is assigned to exactly one center
def assignment_rule(model, i):
    return sum(model.x[i, j] for j in centers) == 1
model.assignment = Constraint(offices, rule=assignment_rule)

# A center must be open if any office is assigned to it
def center_open_rule(model, j):
    return sum(model.x[i, j] for i in offices) <= len(offices) * model.y[j]
model.center_open = Constraint(centers, rule=center_open_rule)

# Only one center is selected
def single_center_rule(model):
    return sum(model.y[j] for j in centers) == 1
model.single_center = Constraint(rule=single_center_rule)

# A center can be selected only if its capacity is greater than 1831
def capacity_selection_rule(model, j):
    return model.y[j] <= (1 if capacities[j] > 1831 else 0)
model.capacity_selection = Constraint(centers, rule=capacity_selection_rule)

In [ ]:
# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp2rfqx05r.glpk.raw --wglp /tmp/tmp4bikl7mx.glpk.glp --cpxlp
 /tmp/tmpd995f63u.pyomo.lp
Reading problem data from '/tmp/tmpd995f63u.pyomo.lp'...
/tmp/tmpd995f63u.pyomo.lp:194659: warning: lower bound of variable 'x2' redefined
/tmp/tmpd995f63u.pyomo.lp:194659: warning: upper bound of variable 'x2' redefined
1885 rows, 47658 columns, 95342 non-zeros
47658 integer variables, all of which are binary
242317 lines were read
Writing problem data to '/tmp/tmp4bikl7mx.glpk.glp'...
192767 lines were written
GLPK Integer Optimizer 5.0
1885 rows, 47658 columns, 95342 non-zeros
47658 integer variables, all of which are binary
Preprocessing...
1851 rows, 32994 columns, 65988 non-zeros
32994 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.832e+03  ratio =  1.832e+03
GM: min|aij| =  9.711e-01  max|aij| =  1.030e+00  ratio =  1.060e+00
EQ: min|aij| =  9.430e-01  max|

In [ ]:
# Display results
print("Optimal Objective Value:", model.objective())

# Display selected center
selected_centers = [j for j in centers if model.y[j].value == 1]
print("Selected Airport:", selected_centers)

Optimal Objective Value: 12601897.144842772
Selected Airport: ['ATL5']
